[[source]](../api/alibi.confidence.linearity_measure.rst)

# Linearity measure

## Overview

Machine learning models include in general linear and non-linear operations: neural networks may include several layers consisting of linear algebra operations followed by non-linear activation functions, while models based on decision trees are by nature highly non-linear. The linearity measure function and class provide an operational definition for the amount of non-linearity of a map acting on vector spaces. Roughly speaking, the amount of non-linearity of the map is defined based on how much the output of the map applied to a linear superposition of input vectors differs from the linear superposition of the map's outputs for each individual vector. In the context of supervised learning, this definition is immediately applicable to machine learning models, which are fundamentally maps from a input vector space (the feature space) to an output vector space that may represent probabilities (for classification models) or actual values of quantities of interest (for regression models).

Given an input vector space $V$, an output vector space $W$ and a map $M: V \rightarrow W$,
the amount of non-linearity of the map $M$ in a region $\beta$ of the input space $V$ and relative to some coefficients $\alpha(\vec{v})$ is defined as
\begin{equation}\label{integral}
  L_{\beta, \alpha}^{(M)} = \left\| \int_{\beta} \alpha(\vec{v}) M(\vec{v}) d\vec{v} -
  M\left(\int_{\beta}\alpha(\vec{v})\vec{v}d\vec{v} \right) \right\|,
\end{equation}
where $\vec{v} \in V$ and $\|\cdot\|$ denotes the norm of a vector. 
If we consider a finite number of vectors, the amount of non-linearity can be defined as
\begin{equation}\label{discrete}
  L_{\beta, \alpha}^{(M)} = \left\| \sum_{i}^N \alpha_{i} M(\vec{v}_i)  -
  M\left(\sum_i^N \alpha_i\vec{v}_i \right) \right\|,
\end{equation}
where, with an abuse of notation,  $\beta$ is no longer a continuous region in
the input space but a collection of input vectors $\{\vec{v}_i\}$ and $\alpha$ is no longer a function but a collection of real coefficients $\{\alpha_i \}$ with $i \in \{0, ..., N\}$. Note that \cref{discrete} may be interpreted as an approximation of the quantity defined in \cref{integral}, where the vectors $\{v_i\}$ are sampled uniformly in the region $\beta$.

## Application to machine learning models

In supervised learning, a model can be considered as a function $M$  mapping vectors from the input space (feature vectors) to vectors in the output space. The output space may represents probabilities in the case of a classification model or values of the target quantities in the case of a regression model. The definition of the linearity measure given above can be applied to the case of a regression model (either a single target regression or a multi target regression) in a straightforward way.

In case of a classifier, let us denote by $\vec{a}$ the logits vector of the model such that the probabilities of the model $M$ are given by $softmax(\vec{a}).$ Since the activation function of the last layer is usually highly non-linear, it is convenient to apply the definition of linearity given above to the logits vector $\vec{a}.$

In the "white box" scenario, in which we have access to the internal architecture of the model,  the vector $\vec{a}$ is accessible and the amount of non-linearity can be calculated immediately. On the other hand,  if the only accessible quantities are the output probabilities (the "black box" scenario), we need to invert the last layer's activation function in order to retrieve $\vec{a}.$ In other words, that means defining a new map $M^\prime = f^{-1} \circ M(\vec{v})$ and considering $L_{\beta, \alpha}^{(M^\prime)}$ as a measure of the non-linearity of the model.  

The activation function of the last layer is usually a sigmoid function for binary classification tasks or a softmax function for multi-class classification. 
The inversion of the sigmoid function does not present any particular challenge, and the map $M^\prime$ can be written as
\begin{equation}
  M^\prime = -\log \circ \Bigg(\frac{1-M(\vec{v})}{M(\vec{v})}\Bigg).
\end{equation}
On the other hand, the softmax probabilities $\vec{p}$ are defined in terms of the vector $\vec{a}$ as $p_j = e^{a_j}/\sum_j{e^{a_j}},$ where $a_j$ are the components of $\vec{a}$. The inverse $softmax^{-1}$ of the softmax function is thus defined up to a constant $C$ which does not depend on $j$ but might depend on the input vector $\vec{v}.$   The inverse map $M^\prime = softmax^{-1} \circ M(\vec{v})$ is then given by:
\begin{equation}\label{MInvSoft}
M^\prime = \log \circ M(\vec{v}) + C(\vec{v}),
\end{equation}
where $C(\vec{v})$ is an arbitrary constant depending in general on the input vector $\vec{v}.$

Since in the black box scenario it is not possible to assess the value of the constants $C$'s, henceforth we will ignore them and define the amount of  non-linearity of a machine learning model whose output is a probability distribution as
\begin{equation}\label{ConstantGaugeLin}
L_{\beta, \alpha}^{(\log \circ M)} = \left\| \sum_{i}^N \alpha_{i} \log \circ M(\vec{v}_i)  -
  \log \circ M\left(\sum_i^N \alpha_i\vec{v}_i \right)\right\|.
\end{equation}
It must be noted that the quantity in \cref{ConstantGaugeLin} may in general be different from the "actual" amount of non-linearity of the model, i.e. the quantity calculated by accessing the activation vectors $\vec{a}$ directly.
However, assuming that $C(\vec{v})= C$  $\forall i$ and $\sum_i \alpha_i = 1,$ it can be shown (see \Cref{app:gauge}) that  $L_{\beta, \alpha}^{M^\prime}$ does not depend on $C$ and is given by
\cref{ConstantGaugeLin}. A more detailed discussion about the role played by the constants $C$ can be found in \Cref{app:gauge}. 

\begin{figure}
    \begin{subfigure}[t]{0.45\textwidth}
        \includegraphics[width=1.0\linewidth]{Lmodels.pdf} 
        \caption{Model comparison}\label{fig:lmodels}
        %\includegraphics[width=\linewidth]{Logistic.png} 
        %\caption{Logistic regression} \label{fig:logistic}
    \end{subfigure}
    \hfill
    \begin{subfigure}[t]{0.45\textwidth}
        \includegraphics[width=\linewidth]{xgboost.pdf} 
        \caption{Xgboost classifier} \label{fig:xgboost}
    \end{subfigure}
    \caption{(a) Comparison of the linearity measure $L$ averaged over the whole features space for various models trained on the iris dataset: random forest (RF), xgboost (XB), support vector machine (SM), neural network (NN) and logistic regression (LR). Note that the scale of the x axis is logarithmic. (b) Decision boundaries (left panel) and linearity measure (right panel) for a xgboost classifier in features space. The x and y axis in the plots represent the sepal length and the sepal width, respectively.  Different colours correspond to different predicted classes. The markers represents the data points in the training set.}
\end{figure}


## Usage